In [3]:

import os
import json
import pandas as pd
import numpy as np
import pickle as pkl
import seaborn as sns
import gensim
from gensim.test.utils import common_texts
import torch

import torch.nn as nn
import torch.nn.functional as F
# load config
with open('config.json', 'r') as f:
    config = json.load(f)
cwd = os.getcwd()
os.chdir(config['REPODIR'])
import Utils as U
from Corpus import Corpus
os.chdir(cwd)

from collections import Counter, defaultdict
import itertools


from tqdm.auto import tqdm, trange
from collections import Counter
import random
from torch import optim
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

from torch.utils.tensorboard import SummaryWriter

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="dark")
import pickle as pkl
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
import nltk
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate

torch.set_default_dtype(torch.float32)

from torch.utils.data.dataloader import default_collate
from torch.utils.tensorboard import SummaryWriter
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
kwargs = {'num_workers': 4, 'pin_memory': True} \
         if (device == "cuda:0" or device == 'mps') else {}
collate_func = lambda x: tuple(x_.to(device) for x_ in default_collate(x)) \
               if device != "cpu" else default_collate

# with open('embedding_data.pkl', 'rb') as f:
#     embed = pkl.load(f)

# embed_df = pd.DataFrame(embed)


# n_classes = embed_df.author_id.nunique()

# from sklearn.preprocessing import OneHotEncoder
# label_encoder=OneHotEncoder(sparse_output=False)

# # -


# X = embed_df['vectors'] # Word Embeddings

# +
test_size = 0.2
val_size = 0.2
random_state =699


# +
with open('sentence_embed.pkl', 'rb') as f:
    embed = pkl.load(f)

embed_df = pd.DataFrame(embed)

data = U.load_file('data_vFFF.pkl', 'pkl', config['DATADIR'])


In [7]:
embed_df[2][0].shape

(100,)

In [2]:

embed_df = pd.DataFrame(data)

embed_df['join_text'] = embed_df['text'].apply(' '.join)
embed_df = embed_df[['author_id', 'text', 'join_text', 'passage_key']]


In [225]:
import torch
torch.cuda.empty_cache()

In [226]:

n_classes = embed_df.author_id.nunique()

from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
y= label_encoder.fit_transform(embed_df['author_id'].to_numpy(dtype='int32').reshape(-1,1))
X = embed_df['passage_key']

X_train, X_test, y_train, y_test = U.train_test_split(X, y, test_size=test_size,
                                                        random_state=random_state,
                                                        stratify=y)

# Split train set into train and validation sets
X_train, X_val, y_train, y_val = U.train_test_split(X_train, y_train, test_size=val_size/(1-test_size),
                                                    random_state=random_state,
                                                    stratify=y_train)



/home/srtoner/.local/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [227]:
embed_df['label'] = y

train_set = set(X_train)
val_set = set(X_val)
test_set = set(X_test)

In [228]:
train = embed_df[embed_df['passage_key'].apply(lambda S: S in train_set)]
val = embed_df[embed_df['passage_key'].apply(lambda S: S in val_set)]
test = embed_df[embed_df['passage_key'].apply(lambda S: S in test_set)]

In [202]:
train.to_csv('train.csv', index=False)
val.to_csv('val.csv', index=False)
test.to_csv('test.csv', index=False)

In [203]:
train

,author_id,text,join_text,passage_key,label
0,5049,[Croft’s next words explained it to be Mr Went...,Croft’s next words explained it to be Mr Wentw...,PG105_2371,115
1,5049,[name and livery included; but I will not pret...,name and livery included; but I will not prete...,PG105_7471,115
3,5049,"[, “Ah! Miss Anne, if it had pleased Heaven to...","“Ah! Miss Anne, if it had pleased Heaven to s...",PG105_2871,115
4,5049,"[fairly off, therefore, before she began to ta...","fairly off, therefore, before she began to tal...",PG105_7921,115
5,5049,"[, This decision checked Mary’s eagerness, and...","This decision checked Mary’s eagerness, and s...",PG105_5121,115
...,...,...,...,...,...
78583,22568,"[from whom, do you think? From Lilly Larnard, ...","from whom, do you think? From Lilly Larnard, t...",PG53940_3241,180
78585,22568,"[the upper one, somewhat after the manner of t...","the upper one, somewhat after the manner of th...",PG53940_4091,180
78587,22568,"[And such a supper! Our host presided, and whi...","And such a supper! Our host presided, and whil...",PG53940_2441,180
78588,22568,"[himself, and you will lose him. Keep him in s...","himself, and you will lose him. Keep him in sl...",PG53940_4241,180


In [229]:
from datasets import Dataset

In [230]:
ds = {
      'train' :  Dataset.from_csv('train.csv'),
      'val' :  Dataset.from_csv('val.csv'),
      'test' :  Dataset.from_csv('test.csv'),
      }

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/srtoner/.cache/huggingface/datasets/csv/default-bff310e37cdca341/0.0.0. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/srtoner/.cache/huggingface/datasets/csv/default-a3cfc9b9446bcb41/0.0.0. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/srtoner/.cache/huggingface/datasets/csv/default-4ead8133ebe384b5/0.0.0. Subsequent calls will reuse this data.


In [231]:
embed_df.columns

Index(['author_id', 'text', 'join_text', 'passage_key', 'label'], dtype='object')

In [232]:
train.to_csv('train.csv', index=False)
val.to_csv('val.csv', index=False)
test.to_csv('test.csv', index=False)

train

,author_id,text,join_text,passage_key,label
0,5049,[Croft’s next words explained it to be Mr Went...,Croft’s next words explained it to be Mr Wentw...,PG105_2371,115
1,5049,[name and livery included; but I will not pret...,name and livery included; but I will not prete...,PG105_7471,115
3,5049,"[, “Ah! Miss Anne, if it had pleased Heaven to...","“Ah! Miss Anne, if it had pleased Heaven to s...",PG105_2871,115
4,5049,"[fairly off, therefore, before she began to ta...","fairly off, therefore, before she began to tal...",PG105_7921,115
5,5049,"[, This decision checked Mary’s eagerness, and...","This decision checked Mary’s eagerness, and s...",PG105_5121,115
...,...,...,...,...,...
78583,22568,"[from whom, do you think? From Lilly Larnard, ...","from whom, do you think? From Lilly Larnard, t...",PG53940_3241,180
78585,22568,"[the upper one, somewhat after the manner of t...","the upper one, somewhat after the manner of th...",PG53940_4091,180
78587,22568,"[And such a supper! Our host presided, and whi...","And such a supper! Our host presided, and whil...",PG53940_2441,180
78588,22568,"[himself, and you will lose him. Keep him in s...","himself, and you will lose him. Keep him in sl...",PG53940_4241,180


In [233]:
ds1

{'train': Dataset({
     features: ['label', 'input_ids', 'attention_mask'],
     num_rows: 47154
 }),
 'val': Dataset({
     features: ['label', 'input_ids', 'attention_mask'],
     num_rows: 15718
 }),
 'test': Dataset({
     features: ['label', 'input_ids', 'attention_mask'],
     num_rows: 15718
 })}

In [234]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers.integrations import TensorBoardCallback
from transformers.trainer_callback import EarlyStoppingCallback
from torch.utils.data import DataLoader


ds1 = {}

BASE_MODEL = "microsoft/MiniLM-L12-H384-uncased"
# BASE_MODEL = "allenai/longformer-base-4096"
# BASE_MODEL = "lreN5bs16" # Learning Rate 2e-5, batch size 16
LEARNING_RATE = 2e-5
MAX_LENGTH = 256
BATCH_SIZE = 16
EPOCHS = .1

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, 
                                                           num_labels=n_classes,
                                                           ignore_mismatched_sizes=True)

def preprocess_function(examples, test = False):
    # if not test:
    label = examples["label"] 
    examples = tokenizer(examples["join_text"],
                        truncation=True, 
                        padding="max_length",
                        max_length=MAX_LENGTH,
                        return_tensors='pt')
    
    for key in examples:
        examples[key] = examples[key].squeeze(0)
  
    # if not test:
    examples["label"] = torch.IntTensor([label])
    examples = examples.to(device)
    return examples

for split in ds:
    ds1[split] = ds[split].map(preprocess_function, 
                                remove_columns=['author_id', 'text', 'passage_key','join_text', 'label'])

    ds1[split].set_format('pt')

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', '

Map:   0%|          | 0/47154 [00:00<?, ? examples/s]

Map:   0%|          | 0/15718 [00:00<?, ? examples/s]

Map:   0%|          | 0/15718 [00:00<?, ? examples/s]

In [235]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="output/",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    logging_steps = 1,
    num_train_epochs=EPOCHS,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="f1",
    greater_is_better=True,
    load_best_model_at_end=True,
    weight_decay=0.01,
    report_to = 'tensorboard'
)

early_stop = EarlyStoppingCallback(1, 0.01)
tb = TensorBoardCallback()


from transformers import Trainer
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        loss_fct = nn.functional.cross_entropy
        loss = loss_fct(logits.view(-1, n_classes), labels.view(-1), average = 'weighted')
        return (loss, outputs) if return_outputs else loss
    

In [236]:
from sklearn.metrics import f1_score

def compute_metrics_for_classification(eval_pred):
    predictions, labels = eval_pred
    labels = labels.reshape(-1, 1)
    print(labels)
    print(type(predictions))
    
    predicted_class = predictions.argmax(axis=1)
    print(predicted_class)
    f1 = f1_score(labels, predicted_class)
    print(f"F1: {f1}")
    
    return {"F1": f1}

In [237]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=ds1["train"],
    eval_dataset=ds1["val"],
    compute_metrics=compute_metrics_for_classification,
)

In [238]:
trainer.train()

/home/srtoner/.local/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 194.00 MiB (GPU 0; 15.78 GiB total capacity; 14.54 GiB already allocated; 70.69 MiB free; 14.75 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer.evaluate()

In [138]:
n_classes

1

prediction = trainer.predict(ds['validation'])

In [ ]:
trainer.save_model('hf_model')